In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-29,France,128003.0,0.0,24113.0,0.0,0.0,0.0
0,2020-04-29,China,84040.0,0.0,4669.0,0.0,0.0,0.0
0,2020-04-29,Italy,203170.0,0.0,27679.0,0.0,0.0,0.0
0,2020-04-29,Spain,211977.0,0.0,23782.0,0.0,0.0,0.0
0,2020-04-29,United States,1033214.0,0.0,60038.0,0.0,0.0,0.0
0,2020-04-29,World,3119609.0,0.0,221290.0,0.0,0.0,0.0
0,2020-04-29,United Kingdom,165163.0,0.0,22039.0,0.0,0.0,0.0
0,2020-04-29,Germany,158505.0,0.0,6303.0,0.0,0.0,0.0
0,2020-04-29,Iran,93577.0,0.0,5968.0,0.0,0.0,0.0
0,2020-04-29,Turkey,116197.0,0.0,3077.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-30,France,128003.0,128442.0,24113.0,24087.0,-439.0,26.0
0,2020-04-30,China,84040.0,83944.0,4669.0,4637.0,96.0,32.0
0,2020-04-30,Italy,203170.0,203591.0,27679.0,27682.0,-421.0,-3.0
0,2020-04-29,Spain,211977.0,212917.0,23782.0,24275.0,-940.0,-493.0
0,2020-04-30,United States,1033214.0,1039909.0,60038.0,60966.0,-6695.0,-928.0
0,2020-04-30,World,3119609.0,3130800.0,221290.0,227771.0,-11191.0,-6481.0
0,2020-04-30,United Kingdom,165163.0,165221.0,22039.0,26097.0,-58.0,-4058.0
0,2020-04-30,Germany,158505.0,159119.0,6303.0,6288.0,-614.0,15.0
0,2020-04-30,Iran,93577.0,93657.0,5968.0,5957.0,-80.0,11.0
0,2020-04-30,Turkey,116197.0,117589.0,3077.0,3801.0,-1392.0,-724.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")